# Hannum

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.Hannum)

class Hannum(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        return x



In [3]:
model = pya.models.Hannum()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'hannum'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2013
model.metadata["approved_by_author"] = '⌛'
model.metadata["citation"] = "Hannum, Gregory, et al. \"Genome-wide methylation profiles reveal quantitative views of human aging rates.\" Molecular cell 49.2 (2013): 359-367.",
model.metadata["doi"] = "https://doi.org/10.1016/j.molcel.2012.10.016"
model.metadata["research_only"] = None
model.metadata["notes"] = None

## Download clock dependencies

#### Download directly with curl

In [5]:
supplementary_url = "https://ars.els-cdn.com/content/image/1-s2.0-S1097276512008933-mmc2.xlsx"
supplementary_file_name = "coefficients.xlsx"
os.system(f"curl -o {supplementary_file_name} {supplementary_url}")

0

## Load features

#### From Excel file

In [6]:
df = pd.read_excel('coefficients.xlsx')
df['feature'] = df['Marker']
df['coefficient'] = df['Coefficient']
model.features = df['feature'].tolist()

## Load weights into base model

In [7]:
weights = torch.tensor(df['coefficient'].tolist()).unsqueeze(0)
intercept = torch.tensor([0.0])

#### Linear model

In [8]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [9]:
model.reference_values = None

## Load preprocess and postprocess objects

In [10]:
model.preprocess_name = None
model.preprocess_dependencies = None

In [11]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [12]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '⌛',
 'citation': ('Hannum, Gregory, et al. "Genome-wide methylation profiles '
              'reveal quantitative views of human aging rates." Molecular cell '
              '49.2 (2013): 359-367.',),
 'clock_name': 'hannum',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.1016/j.molcel.2012.10.016',
 'notes': None,
 'research_only': None,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2013}
reference_values: None
preprocess_name: None
preprocess_dependencies: None
postprocess_name: None
postprocess_dependencies: None
features: ['cg20822990', 'cg22512670', 'cg25410668', 'cg04400972', 'cg16054275', 'cg10501210', 'cg09809672', 'ch.2.30415474F', 'cg22158769', 'cg02085953', 'cg06639320', 'cg22454769', 'cg24079702', 'cg23606718', 'cg22016779', 'cg04474832', 'cg03607117', 'cg07553761', 'cg00481951', 'cg25478614', 'cg25

## Basic test

In [13]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 78.6234],
        [  7.5513],
        [172.7843],
        [ 43.1732],
        [145.3521],
        [ 79.6871],
        [ 75.0541],
        [ 41.7890],
        [-12.6555],
        [-78.9716]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [14]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [15]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: coefficients.xlsx
